In [85]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [86]:
import pandas as pd
import numpy as np
from pydantic import BaseModel
import sys
import os
from lightning.pytorch.loggers import MLFlowLogger
from loguru import logger
from load_dotenv import load_dotenv
import time
import json
import torch
from torch.utils.data import DataLoader
import lightning as L
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint
import mlflow

sys.path.insert(0, "..")

from src.utils.embedding_id_mapper import IDMapper
from src.algo.gSASRec.model import SASRec
from src.algo.gSASRec.dataset import SASRecDataset
from src.algo.gSASRec.trainer import SASRecLitModule
from src.eval.utils import create_rec_df, create_label_df, merge_recs_with_target
from src.eval.log_metrics import log_ranking_metrics, log_classification_metrics

In [87]:
load_dotenv(override = True)

False

In [88]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "first-attempt"
    run_name: str = f"018-sasrec"
    notebook_persit_dp: str = None
    
    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"
    group_name: str = "seq-modelling"

    top_K: int = 100
    top_k: int = 10

    batch_size: int = 256
    lr: float = 0.001
    l2_emb: float = 0.0001
    early_stopping_patience: int = 10
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    num_epochs: int = 100

    # SASrec specific
    max_len: int = 10
    dropout: float = 0.3
    hidden_units: int = 128
    num_blocks: int = 1
    num_heads: int = 2
    num_workers: int = 4
    pad_token: int = 4817
    # seq_length: int = 10
    
    train_data_fp: str = os.path.abspath("../data_for_ai/interim/train_sample_interactions_16407u_neg_seq.parquet")
    val_data_fp: str = os.path.abspath("../data_for_ai/interim/val_sample_interactions_16407u_neg_seq.parquet")

    def init(self):
        self.notebook_persit_dp = os.path.abspath(f"data/{self.experiment_name}/{self.run_name}")

        if not (mlflow_uri := os.environ.get("MLFLOW_TRACKING_URI")):
            self.log_to_mlflow = False
            logger.warning("MLFlow is not enabled. Turn off tracking to Mlflow.")

        if self.log_to_mlflow:
            logger.info(
                f"Setting up Mlflow experiment: {self.experiment_name}, run_name: {self.run_name}"
            )

            self._mlf_logger = MLFlowLogger(
                experiment_name=self.experiment_name,
                run_name=self.run_name,
                tracking_uri=mlflow_uri,
                log_model=True,
            )

        if not self.testing:
            os.makedirs(self.notebook_persit_dp, exist_ok=True)
        return self
    
args = Args().init()
print(args.model_dump_json(indent=2))

2025-04-30 14:53:44.164 | WARNING  | __main__:init:42 - MLFlow is not enabled. Turn off tracking to Mlflow.


{
  "testing": false,
  "log_to_mlflow": false,
  "experiment_name": "first-attempt",
  "run_name": "018-sasrec",
  "notebook_persit_dp": "c:\\Users\\Trieu\\OneDrive\\Desktop\\recsys\\real_time_recsys\\notebooks\\data\\first-attempt\\018-sasrec",
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "group_name": "seq-modelling",
  "top_K": 100,
  "top_k": 10,
  "batch_size": 256,
  "lr": 0.001,
  "l2_emb": 0.0001,
  "early_stopping_patience": 10,
  "device": "cpu",
  "num_epochs": 100,
  "max_len": 10,
  "dropout": 0.3,
  "hidden_units": 128,
  "num_blocks": 1,
  "num_heads": 2,
  "num_workers": 4,
  "pad_token": 4817,
  "train_data_fp": "c:\\Users\\Trieu\\OneDrive\\Desktop\\recsys\\real_time_recsys\\data_for_ai\\interim\\train_sample_interactions_16407u_neg_seq.parquet",
  "val_data_fp": "c:\\Users\\Trieu\\OneDrive\\Desktop\\recsys\\real_time_recsys\\data_for_ai\\interim\\val_sample_interactions_16407u_neg_seq.parquet"
}


In [89]:
train_df = pd.read_parquet(args.train_data_fp)
# convert train_df["rating"] to 1 if rating > 0 else 0
train_df[args.rating_col] = train_df[args.rating_col].apply(lambda x: 1 if x > 0 else 0)            
# train_df["item_sequence"] = train_df["item_sequence"].apply(lambda x: list(x))

val_df = pd.read_parquet(args.val_data_fp)
# convert val_df["rating"] to 1 if rating > 0 else 0
val_df[args.rating_col] = val_df[args.rating_col].apply(lambda x: 1 if x > 0 else 0)

assert set(val_df[args.user_col].unique()).issubset(set(train_df[args.user_col].unique())), "Validation users must be present in training users."

assert set(val_df[args.item_col].unique()).issubset(set(train_df[args.item_col].unique())), "Validation items must be present in training items."
assert train_df[args.timestamp_col].max() < val_df[args.timestamp_col].min(), "Validation data must be after training data. Otherwise, its a data contamination problem."

In [90]:
val_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,item_sequence
0,AGSP5XAQPQBUUXZHEZSC65FD7NOQ,B004FV4ROA,1,2020-12-27 00:30:31.146,11295,528,"[1898, 3479, 3908, 1570, 91, 2723, 2962, 106, ..."
1,AGSP5XAQPQBUUXZHEZSC65FD7NOQ,B07KFQFDNB,0,2020-12-27 00:30:31.146,11295,3503,"[3479, 3908, 1570, 91, 2723, 2962, 106, 3557, ..."
2,AEHS7YR7BGGWMZS24H5UR5IP46HQ,B08F1P3BCC,1,2020-12-27 01:44:52.242,1784,3925,"[4319, 3382, 4330, 1173, 1330, 423, 2868, 3167..."
3,AEHS7YR7BGGWMZS24H5UR5IP46HQ,B00HXT8EKE,0,2020-12-27 01:44:52.242,1784,1507,"[3382, 4330, 1173, 1330, 423, 2868, 3167, 1071..."
4,AGAVHCK42EGMVS7DGPRX6HBCUCNQ,B09Q3NR84W,1,2020-12-27 02:25:48.357,9042,4273,"[1311, 1416, 455, 3743, 1823, 2694, 3612, 3462..."
...,...,...,...,...,...,...,...
6953,AEEQZRQBOFHFBFPYBX2BZ5WOI33A,B01A08E70K,0,2022-02-19 16:56:53.030,1396,2441,"[3451, 3827, 1839, 1347, 2504, 2694, 4546, 427..."
6954,AHLN6GKTKZE22AON34YAQXTGK63A,B0C682GZ5X,1,2022-02-19 17:28:55.519,14550,4772,"[2950, 1812, 4735, 4165, 4575, 2440, 607, 4807..."
6955,AHLN6GKTKZE22AON34YAQXTGK63A,B09SWWCN6Q,0,2022-02-19 17:28:55.519,14550,4303,"[1812, 4735, 4165, 4575, 2440, 607, 4807, 374,..."
6956,AEMYBWDN67IB5IBTMHLHN76V4QHQ,B091K4WYD1,1,2022-02-19 22:08:53.253,2446,4086,"[644, 3602, 4569, 1865, 3030, 3653, 3803, 3998..."


In [91]:
train_df.head(3)

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,item_sequence
0,AFZ4EK2LJ655XQKTEUELCARO6RYA,B00002EQCW,1,2003-01-23 03:28:15,8071,4,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."
1,AFZ4EK2LJ655XQKTEUELCARO6RYA,B095JX15XF,0,2003-01-23 03:28:15,8071,4132,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."
2,AFY2C4YOUP2SSMM43HD2L3FIEFZA,B00008SCFL,1,2003-11-25 18:12:09,7935,36,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."


In [92]:
def init_model(n_user, n_items, dropout, hidden_units, num_blocks, num_heads):
    """
    Initialize the model with the given parameters.
    """
    model = SASRec(
        user_num = n_user,
        item_num = n_items,
        dropout_rate = dropout,
        hidden_units = hidden_units,
        num_blocks = num_blocks,
        num_heads = num_heads,
    )
    return model

In [93]:
batch_size = 2
hidden_units = 8
dropout = 0.2
num_blocks = 1
num_heads = 2

# Mock data
user_indices = [0, 0, 1, 2, 2]
item_indices = [0, 1, 2, 3, 4]
timestamps = [0, 1, 2, 3, 4]
ratings = [0, 4, 5, 3, 0]

user_num = len(set(user_indices))
item_num = len(set(item_indices))

train_test_df = pd.DataFrame(
    {
        "user_indice": user_indices,
        "item_indice": item_indices,
        args.timestamp_col: timestamps,
        args.rating_col: ratings,
    }
)

model = init_model(user_num, item_num, dropout,hidden_units, num_blocks, num_heads)

# Example forward pass
model.eval()
user = torch.tensor([[0]])
seq = torch.tensor([[0,0,0,0,0,1,2,3,4,5]])
target_item = torch.tensor([[2]])
predictions = model.predict(user, seq, target_item)
print(predictions)

tensor([[0.6678]], grad_fn=<SigmoidBackward0>)


In [94]:
print(type(train_df["item_sequence"][0]))


<class 'numpy.ndarray'>


In [95]:
train_df["item_indice"].max() + 1

4817

In [96]:
rating_dataset = SASRecDataset(
    train_df, "user_indice", "item_sequence", "item_indice", "rating",args.max_len, args.pad_token, args.timestamp_col, 
)
val_rating_dataset = SASRecDataset(
    val_df, "user_indice", "item_sequence", "item_indice", "rating", args.max_len, args.pad_token, args.timestamp_col, 
)

train_loader = DataLoader(
    rating_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True
)
val_loader = DataLoader(
    val_rating_dataset, batch_size=args.batch_size, shuffle=False, drop_last=False
)

In [97]:
for i in val_loader:
    print(i["user"])
    print(i["sequence"])
    print(i["item"])
    print(i["rating"])
    break

tensor([11295, 11295,  1784,  1784,  9042,  9042,  1542,  1542, 10418, 10418,
         2786,  2786,  6758,  6758,  8761,  8761,  6185,  6185,  1134,  1134,
        11437, 11437, 12823, 12823,  8666,  8666,  6710,  6710,  1590,  1590,
         1590,  1590, 11747, 11747,  2393,  2393, 10403, 10403,  3982,  3982,
         4098,  4098,  7417,  7417,  6394,  6394,  7293,  7293,  5421,  5421,
        14988, 14988,  1574,  1574, 16260, 16260, 10908, 10908,  1861,  1861,
        14374, 14374, 13038, 13038,  8429,  8429,  2164,  2164,  4053,  4053,
         6451,  6451,  1394,  1394, 15498, 15498, 11209, 11209, 14837, 14837,
        15761, 15761, 10092, 10092, 12914, 12914, 13467, 13467,  6383,  6383,
         7731,  7731,  1677,  1677,  7342,  7342,  7824,  7824, 11332, 11332,
         2189,  2189,  5829,  5829,  5942,  5942,   746,   746, 12588, 12588,
         4263,  4263, 13562, 13562, 12115, 12115, 12115, 12115, 12115, 12115,
        12115, 12115,  3016,  3016, 12115, 12115,  7119,  7119, 

In [98]:
for i in train_loader:
    print(i["user"])
    print(i["sequence"])
    print(i["item"])
    print(i["rating"])
    break

tensor([10467,  1335, 15231,  7275,  3898,  4562,  5541, 11889,  2147, 13798,
          910,  3707, 14652,  3270,  6150, 10293, 12893,  8481,   128, 13441,
         9831,  6787, 14815,   766,  9231, 14230,  7491, 11584,  2370,  2604,
          385, 12161, 15306,   525,    36, 12810,  4673, 11598,  2186,  3649,
         4208, 13337,  1492,  3421,   545, 12040, 11131,  8629, 11085,  5735,
         3538,  7841, 10649,  2356, 15362,  5939,  8868,  6931,  4526, 14534,
          510,  5709,  8487,  7530, 11638, 12083,  5026,  7509,  7567, 11060,
         4234,  9355,  8862,  6533, 10592,  2392,  2043,   334,  1524, 10311,
        12264, 16142, 10637, 10580,  3302,  1850,  7688, 15044, 12896, 13075,
        10171,  2176, 12662, 14197,  3413,  3889,  3828,   499,  5215,   515,
        13127,  5726,  1606, 15301, 13868, 13075, 13309,  5655,  3735, 16311,
          908,  6981,  7974,  6800, 10779,  6057, 11875,  8072,  5024,  8479,
         2268,  5353, 14951, 11543,  2077,  7579,  7570, 14287, 

In [99]:
item_indices = train_df[args.item_col].unique()
user_indices = train_df[args.user_col].unique()
n_items = len(item_indices)
n_users = len(user_indices)

logger.info(f"Number of users: {n_users}, Number of items: {n_items}")
model = init_model(n_users, n_items, args.dropout, args.hidden_units, args.num_blocks, args.num_heads)

2025-04-30 14:53:49.224 | INFO     | __main__:<module>:6 - Number of users: 16407, Number of items: 4817


In [100]:
model

SASRec(
  (item_emb): Embedding(4818, 128, padding_idx=4817)
  (pos_emb): Embedding(10, 128)
  (emb_dropout): Dropout(p=0.3, inplace=False)
  (attention_layernorms): ModuleList(
    (0): LayerNorm((128,), eps=1e-08, elementwise_affine=True)
  )
  (attention_layers): ModuleList(
    (0): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
    )
  )
  (forward_layernorms): ModuleList(
    (0): LayerNorm((128,), eps=1e-08, elementwise_affine=True)
  )
  (forward_layers): ModuleList(
    (0): PointWiseFeedForward(
      (conv1): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
      (dropout1): Dropout(p=0.3, inplace=False)
      (relu): ReLU()
      (conv2): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
      (dropout2): Dropout(p=0.3, inplace=False)
    )
  )
  (final_layer): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [101]:
idm_path = os.path.abspath("../data_for_ai/interim/idm_16407u.json")
idm = IDMapper().load(idm_path)
idm.get_user_id(1)

'AE227WAM4NWQPJI33OPN7ZARNNZQ'

## overfit 1 batch

In [102]:
early_stopping = EarlyStopping(
    monitor="val_loss", patience=5, mode="min", verbose=False
)

model = init_model(n_users, n_items, args.dropout, args.hidden_units, args.num_blocks, args.num_heads)
lit_model = SASRecLitModule(
    model,
    log_dir=args.notebook_persit_dp,
    accelerator=args.device,
    lr=args.lr,
    l2_emb=args.l2_emb,
    idm= idm
)

log_dir = f"{args.notebook_persit_dp}/logs/overfit"

# train model
trainer = L.Trainer(
    default_root_dir=log_dir,
    accelerator=args.device if args.device else "auto",
    max_epochs=args.num_epochs,
    overfit_batches=1,
    callbacks=[early_stopping],
)
trainer.fit(
    model=lit_model,
    train_dataloaders=train_loader,
    val_dataloaders=train_loader,
)
logger.info(f"Logs available at {trainer.log_dir}")

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.

  | Name  | Type   | Params | Mode 
-----------------------------------------
0 | model | SASRec | 717 K  | train
-----------------------------------------
717 K     Trainable params
0         Non-trainable params
717 K     Total params
2.871     Total estimated model params size (MB)
20        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\Trieu\AppData\Local\pypoetry\Cache\virtualenvs\hm-scalablerecs-QHnDFvap-py3.11\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:252: UserWarning:

You requested to overfit but enabled val dataloader shuffling. We are turning off the val dataloader shuffling for you.

c:\Users\Trieu\AppData\Local\pypoetry\Cache\virtualenvs\hm-scalablerecs-QHnDFvap-py3.11\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: PossibleUserWarning:

The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.

c:\Users\Trieu\AppData\Local\pypoetry\Cache\virtualenvs\hm-scalablerecs-QHnDFvap-py3.11\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:252: UserWarning:

You requested to overfit but enabled train dataloader shuffling. We are turning off the train dataloader shuffling for you.

c:\Use

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

2025-04-30 14:54:14.557 | INFO     | src.algo.gSASRec.trainer:on_fit_end:4 - Evidently metrics are available at: c:\Users\Trieu\OneDrive\Desktop\recsys\real_time_recsys\notebooks\data\first-attempt\018-sasrec
2025-04-30 14:54:14.560 | INFO     | __main__:<module>:30 - Logs available at c:\Users\Trieu\OneDrive\Desktop\recsys\real_time_recsys\notebooks\data\first-attempt\018-sasrec\logs\overfit\lightning_logs\version_46


In [103]:
early_stopping = EarlyStopping(
    monitor="val_loss", patience=args.early_stopping_patience, mode="min", verbose=False, min_delta=0.0025
)

checkpoint_callback = ModelCheckpoint(
    dirpath=f"{args.notebook_persit_dp}/checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    monitor="val_loss",
    mode="min",
)

model = init_model(n_users, n_items, args.embedding_dim, args.dropout)
lit_model = SASRecLitModule(
    model,
    lr=args.lr,
    l2_emb=args.l2_emb,
    log_dir=args.notebook_persit_dp,
    accelerator=args.device,
    idm= idm
)

log_dir = f"{args.notebook_persit_dp}/logs/run"

# train model
trainer = L.Trainer(
    default_root_dir=log_dir,
    accelerator=args.device if args.device else "auto",
    max_epochs=args.num_epochs,
    callbacks=[early_stopping, checkpoint_callback],
    logger=args._mlf_logger if args.log_to_mlflow else None,
)
trainer.fit(
    model=lit_model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
)

# Change the library as a workaround for the issue in the latest Lightning release
#https://github.com/Lightning-AI/pytorch-lightning/pull/20669/commits/429f732a0528c558e701da7ec01e51c1e2e4f32e

AttributeError: 'Args' object has no attribute 'embedding_dim'

In [ ]:
all_params = [args]

if args.log_to_mlflow:
    run_id = trainer.logger.run_id

    with mlflow.start_run(run_id=run_id):
        for params in all_params:
            params_dict = params.model_dump()
            params_ = dict()
            for k, v in params_dict.items():
                if k == "top_K":
                    k = "top_big_K"
                if k == "top_k":
                    k = "top_small_k"
                params_[f"{params.__repr_name__()}.{k}"] = v
            mlflow.log_params(params_)